# Import libraries

In [1]:
import pymongo
import json
import requests
import pandas as pd
from pprint import pprint

# Create/Connect to a database and a collection

In [2]:
# Connection to Mongo DB
try:
    client = pymongo.MongoClient()
    print ("Connected to MongoDB successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e)

db = client.DSlabel_db
collection = db.DSlabel_collection_final

Connected to MongoDB successfully!!!


In [3]:
print ("Databases:", client.database_names())
print ("Collections:", db.collection_names())
print ('Number of documents:', collection.count())

Databases: ['DSlabel_db', 'admin', 'local']
Collections: ['DSlabel_collection_final']
Number of documents: 65499


# Insert documents into collection

In [ ]:
collection.drop()

In [ ]:
all_products = pd.read_csv('data/DSLD_AllProducts.csv')
DSLD_API = "http://dsld.nlm.nih.gov/dsld/api/label/"

for DSLD_ID in all_products['DSLD ID']:
    try:
        current_url = '{0}{1}'.format(DSLD_API, DSLD_ID)
        print('Fetching data from {}'.format(current_url))
        
        response = requests.get(current_url)
        data = response.json()
        label_id = collection.insert_one(data).inserted_id
        
    except requests.exceptions.RequestException as e:  
        print (e)

In [ ]:
print ("Databases:", client.database_names())
print ("Collections:", db.collection_names())
print ('Number of documents:', collection.count())

# Get the keys of the collection

In [4]:
cursor = collection.aggregate([
          {"$project":{"arrayofkeyvalue":{"$objectToArray":"$$ROOT"}}},
          {"$unwind":"$arrayofkeyvalue"},
          {"$group":{"_id":'null',"allkeys":{"$addToSet":"$arrayofkeyvalue.k"}}}
        ])
pprint(list(cursor))

[{'_id': 'null',
  'allkeys': ['Tracking_History',
              'contacts',
              'LanguaL_Intended_Target_Groups',
              'Brand',
              'statements',
              'ingredients',
              'LanguaL_Product_Type',
              'Serving_Size',
              'LanguaL_Supplement_Form',
              'DSLD_ID',
              'count',
              'Product_Name',
              '_id',
              'Outer_Packaging',
              'Suggested_Use',
              'LanguaL_Dietary_Claim_or_Use',
              'NHANES_ID',
              'success',
              'Date_Entered_into_DSLD',
              'Database',
              'Net_Contents_Quantity',
              'Product_Trademark_Copyright_Symbol',
              'Statement_of_Identity',
              'SKU',
              'DB_Source']}]


In [5]:
pprint(collection.find_one())

{'Brand': '+nuun',
 'DB_Source': 'Version 7.0.1 - October 2017, Rev 2503 (46eaaddd8e99)',
 'DSLD_ID': '40479',
 'Database': 'adsld',
 'Date_Entered_into_DSLD': 'December 29, 2014',
 'LanguaL_Dietary_Claim_or_Use': 'OTHER INGREDIENT- OR CONSTITUENT-RELATED '
                                 'CLAIM OR USE [P0115]',
 'LanguaL_Intended_Target_Groups': 'HUMAN CONSUMER, FOUR YEARS AND ABOVE '
                                   '[P0250]',
 'LanguaL_Product_Type': 'MULTI-VITAMIN AND MULTI-MINERAL SUPPLEMENT [A1315]',
 'LanguaL_Supplement_Form': 'TABLET [E0155]',
 'NHANES_ID': '',
 'Net_Contents_Quantity': '15.0 Tab(s)Alternate 58.0 Gram(s)',
 'Outer_Packaging': 'Not Present',
 'Product_Name': '+nuun All Day Blueberry Pomegranate',
 'Product_Trademark_Copyright_Symbol': '',
 'SKU': 'Not Present',
 'Serving_Size': ' 1.0 Tablet(s)',
 'Statement_of_Identity': '',
 'Suggested_Use': '',
 'Tracking_History': 'Date - Entered into DSLD: December 29, 2014',
 '_id': ObjectId('5a98485c36745709af7d057a'),


# create a subset of the collection

In [6]:
cursor = collection.find({}, {'DSLD_ID': 1,
                              'Brand': 1, 
                              'Product_Name': 1,
                              'LanguaL_Intended_Target_Groups': 1,
                              'LanguaL_Dietary_Claim_or_Use': 1,
                              'statements': 1,
                              'ingredients': 1,
                              'LanguaL_Product_Type': 1,
                              'LanguaL_Supplement_Form': 1,
                              'Serving_Size': 1,
                              'success': 1,
                              '_id':0
                             })

In [7]:
print('{} documents in collection'.format(cursor.count()))

65499 documents in collection


# Convert to pandas Dataframe and explore

In [8]:
# Expand the cursor and construct the DataFrame
DS_label_df = pd.DataFrame(list(cursor))

In [9]:
DS_label_df.shape

(65499, 11)

In [10]:
DS_label_df.columns

Index(['Brand', 'DSLD_ID', 'LanguaL_Dietary_Claim_or_Use',
       'LanguaL_Intended_Target_Groups', 'LanguaL_Product_Type',
       'LanguaL_Supplement_Form', 'Product_Name', 'Serving_Size',
       'ingredients', 'statements', 'success'],
      dtype='object')

In [11]:
DS_label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65499 entries, 0 to 65498
Data columns (total 11 columns):
Brand                             65499 non-null object
DSLD_ID                           65499 non-null object
LanguaL_Dietary_Claim_or_Use      65499 non-null object
LanguaL_Intended_Target_Groups    65499 non-null object
LanguaL_Product_Type              65499 non-null object
LanguaL_Supplement_Form           65499 non-null object
Product_Name                      65499 non-null object
Serving_Size                      65499 non-null object
ingredients                       65499 non-null object
statements                        65499 non-null object
success                           65499 non-null bool
dtypes: bool(1), object(10)
memory usage: 5.1+ MB


In [12]:
DS_label_df.head()

,Brand,DSLD_ID,LanguaL_Dietary_Claim_or_Use,LanguaL_Intended_Target_Groups,LanguaL_Product_Type,LanguaL_Supplement_Form,Product_Name,Serving_Size,ingredients,statements,success
0,+nuun,40479,OTHER INGREDIENT- OR CONSTITUENT-RELATED CLAIM...,"HUMAN CONSUMER, FOUR YEARS AND ABOVE [P0250]",MULTI-VITAMIN AND MULTI-MINERAL SUPPLEMENT [A1...,TABLET [E0155],+nuun All Day Blueberry Pomegranate,1.0 Tablet(s),"[{'Ingredient_ID': '7388', 'DSLD_Ingredient_Ca...","[{'Statement': 'UNDER 8 CALORIES', 'Statement_...",True
1,+nuun,40480,OTHER INGREDIENT- OR CONSTITUENT-RELATED CLAIM...,"HUMAN CONSUMER, FOUR YEARS AND ABOVE [P0250]",MULTI-VITAMIN AND MULTI-MINERAL SUPPLEMENT [A1...,TABLET [E0155],+nuun All Day Grape Raspberry,1.0 Tablet(s),"[{'Ingredient_ID': '7388', 'DSLD_Ingredient_Ca...","[{'Statement': 'ALL NATURAL, ZERO SUGAR', 'Sta...",True
2,+nuun,40464,OTHER INGREDIENT- OR CONSTITUENT-RELATED CLAIM...,"HUMAN CONSUMER, FOUR YEARS AND ABOVE [P0250]",MULTI-VITAMIN AND MULTI-MINERAL SUPPLEMENT [A1...,TABLET [E0155],+nuun All Day Tangerine Lime,1.0 Tablet(s),"[{'Ingredient_ID': '7388', 'DSLD_Ingredient_Ca...",[{'Statement': 'DRINK YOUR VITAMINS + A B C D...,True
3,+nuun,40438,OTHER INGREDIENT- OR CONSTITUENT-RELATED CLAIM...,"HUMAN CONSUMER, FOUR YEARS AND ABOVE [P0250]",BOTANICAL SUPPLEMENT WITH VITAMIN/MINERAL [A1317],TABLET [E0155],+nuun Natural Hydration Goji Berry Green Tea,16.0 fl. Oz.Alternate 475.0 mL,"[{'Ingredient_ID': '7388', 'DSLD_Ingredient_Ca...",[{'Statement': 'Use only if cap seal is unbrok...,True
4,+nuun,40456,OTHER INGREDIENT- OR CONSTITUENT-RELATED CLAIM...,"HUMAN CONSUMER, FOUR YEARS AND ABOVE [P0250]",BOTANICAL SUPPLEMENT WITH VITAMIN/MINERAL [A1317],TABLET [E0155],+nuun Natural Hydration Tangerine Ginger,1.0 Effervescent Tablet(s),"[{'Ingredient_ID': '7388', 'DSLD_Ingredient_Ca...","[{'Statement': 'by +nuun', 'Statement_Type': '...",True


# Pickle the dataframe

In [ ]:
# write dataframe
DS_label_df.to_csv('data/DietarySupplementLabelDatabase.csv', index=False)
DS_label_df.to_pickle('data/DietarySupplementLabelDatabase.pkl')

In [ ]:
# read dataframe
DS_label_df = pd.read_pickle('data/DietarySupplementLabelDatabase.pkl')